In [1]:
import numpy as np
from glm.families import Gaussian, Bernoulli, Poisson, Gamma
from glm.glm import GLM
from glm.simulation import Simulation

import statsmodels.api as sm
import statsmodels

/Users/matthewdrury/anaconda/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/Users/matthewdrury/anaconda/envs/py36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
N = 10000
X = np.empty(shape=(N, 3))
X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=N)
X[:, 2] = np.random.uniform(size=N)
nu = 1 - 2*X[:, 1] + X[:, 2]

## Linear Model

In [3]:
y = nu + np.random.normal(size=N)
model = GLM(family=Gaussian())
model.fit(X, y)

In [4]:
model.coef_

array([ 0.9448427 , -1.95999816,  1.03653559])

In [5]:
model.coef_covariance_matrix_

array([[  6.86809065e-04,  -5.85460914e-04,  -5.92904909e-04],
       [ -5.85460914e-04,   1.17252920e-03,  -5.55830125e-08],
       [ -5.92904909e-04,  -5.55830125e-08,   1.19191634e-03]])

In [6]:
model.coef_standard_error_

array([ 0.02620704,  0.03424221,  0.03452414])

In [7]:
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.295
Model:                            OLS   Adj. R-squared:                  0.295
Method:                 Least Squares   F-statistic:                     2089.
Date:                Mon, 11 Sep 2017   Prob (F-statistic):               0.00
Time:                        21:49:42   Log-Likelihood:                -14164.
No. Observations:               10000   AIC:                         2.833e+04
Df Residuals:                    9997   BIC:                         2.836e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9448      0.026     36.053      0.0

## Run some simulations off the linear model.

In [8]:
s = Simulation(model)

In [9]:
s.sample(X)

array([[ 1.28728337,  0.33289998,  0.92487895, ..., -0.51618203,
         0.04111571,  2.03737834],
       [-0.61612043,  0.13385134,  2.83120743, ..., -0.30199972,
        -0.79479901,  0.39789668],
       [ 0.72198974, -1.16442492,  1.99196898, ..., -1.1489727 ,
        -0.62900916,  0.3819916 ],
       ..., 
       [ 0.14984226,  2.21988792,  0.35897445, ..., -0.19421481,
         1.86637245,  0.11179883],
       [ 0.63805767,  2.3051978 ,  2.0584596 , ..., -0.63083674,
         1.6915411 ,  0.97459907],
       [ 1.38382519, -0.3741162 ,  1.14145529, ...,  0.08402882,
        -1.23257719, -1.47523731]])

In [10]:
models = s.parametric_bootstrap(X, n_sim=10)
for model in models:
    print(model.coef_)

[ 0.95344796 -1.96423232  1.04301144]
[ 0.89767002 -1.89585451  1.03593506]
[ 0.97160611 -2.03299058  1.05701442]
[ 0.93765236 -1.99102363  1.06143271]
[ 0.97356691 -1.98085228  1.00870744]
[ 0.9348543  -1.92503744  1.0282268 ]
[ 0.95273462 -1.9835889   1.00119733]
[ 0.92528941 -1.95512238  1.00807083]
[ 0.91685828 -1.92017611  1.04261013]
[ 0.94781326 -1.94359565  1.03430831]


In [11]:
models = s.non_parametric_bootstrap(X, y, n_sim=10)
for model in models:
    print(model.coef_)

[ 0.93512214 -1.97638435  1.04210078]
[ 0.90916096 -1.92721013  1.01932087]
[ 0.96254168 -1.96276683  1.01793319]
[ 0.99777005 -2.01256766  1.00805192]
[ 0.96349935 -1.99652488  1.03958545]
[ 0.94308996 -1.98564392  1.0621469 ]
[ 0.93756372 -1.95792508  1.05527984]
[ 0.90222959 -1.87468415  1.03975222]
[ 0.97776449 -1.99672265  1.01185453]
[ 0.91167987 -1.92691269  1.04754695]


## Linear Model with Sample Weights

In [12]:
sample_weights = np.random.uniform(0, 2, size=N)

In [13]:
model = GLM(family=Gaussian())
model = model.fit(X, y, sample_weights=sample_weights)

In [14]:
model.coef_

array([ 0.93239994, -1.89616789,  1.01662108])

## Logistic Model

In [15]:
p = 1 / (1 + np.exp(-nu))
y_logistic = np.random.binomial(1, p=p, size=N)

In [16]:
model = GLM(family=Bernoulli())
model.fit(X, y_logistic)

In [17]:
model.coef_

array([ 1.0171102 , -1.96325661,  0.94684443])

In [18]:
model.dispersion_

array(1.0)

In [19]:
model.coef_covariance_matrix_

array([[ 0.0032339 , -0.0029296 , -0.00250258],
       [-0.0029296 ,  0.00581565, -0.00037811],
       [-0.00250258, -0.00037811,  0.0056142 ]])

In [20]:
model.coef_standard_error_

array([ 0.05686735,  0.0762604 ,  0.07492798])

In [21]:
mod = sm.Logit(y_logistic, X)
res = mod.fit()
print(res.summary())

Optimization terminated successfully.
         Current function value: 0.623820
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Mon, 11 Sep 2017   Pseudo R-squ.:                 0.06470
Time:                        21:49:42   Log-Likelihood:                -6238.2
converged:                       True   LL-Null:                       -6669.7
                                        LLR p-value:                3.929e-188
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0171      0.057     17.886      0.000       0.906       1.129
x1            -1.9633      0.

In [22]:
s = Simulation(model)

In [23]:
s.sample(X, n_sim=10)

array([[ 0.,  1.,  1., ...,  1.,  0.,  0.],
       [ 1.,  1.,  0., ...,  0.,  0.,  1.],
       [ 0.,  1.,  1., ...,  1.,  1.,  0.],
       ..., 
       [ 1.,  1.,  1., ...,  0.,  0.,  1.],
       [ 0.,  1.,  1., ...,  0.,  0.,  1.],
       [ 0.,  1.,  0., ...,  0.,  0.,  1.]])

In [24]:
for model in s.parametric_bootstrap(X, n_sim=10):
    print(model.coef_)

[ 1.03396928 -1.94073484  0.86886517]
[ 1.08320339 -2.07276846  0.8941195 ]
[ 1.051676   -1.9909684   0.91835684]
[ 1.03286647 -1.88940083  0.84943943]
[ 1.03808526 -1.99804694  0.93431155]
[ 1.02159681 -1.91795833  0.91778819]
[ 1.02702098 -1.98645933  0.96355766]
[ 1.06989729 -2.01122924  0.93480326]
[ 0.86205479 -1.8616221   1.06135952]
[ 0.94031612 -1.89618373  0.98906941]


In [25]:
for model in s.non_parametric_bootstrap(X, y_logistic, n_sim=10):
    print(model.coef_)

[ 1.11306968 -2.00309832  0.79211228]
[ 1.05088821 -1.91454067  0.93678611]
[ 0.97725535 -1.84797033  0.94971403]
[ 0.96132623 -1.91547349  1.04409077]
[ 0.94992351 -1.93756706  1.00671207]
[ 0.96758563 -1.92788782  0.9665649 ]
[ 0.98747473 -1.94482768  0.93850774]
[ 1.13325277 -2.12416241  0.95059659]
[ 1.01781512 -2.09842983  1.02840433]
[ 1.04127508 -1.9362662   0.91700516]


## Poission Model

In [26]:
mu = np.exp(nu)
y_poisson = np.random.poisson(lam=mu, size=N)

In [27]:
model = GLM(family=Poisson())
model.fit(X, y_poisson)

In [28]:
model.coef_

array([ 0.97955491, -1.97310256,  0.99957952])

In [29]:
model.coef_covariance_matrix_

array([[  3.43518078e-04,  -2.43028083e-04,  -3.62884496e-04],
       [ -2.43028083e-04,   7.15362672e-04,  -3.42171090e-06],
       [ -3.62884496e-04,  -3.42171090e-06,   6.28322610e-04]])

In [30]:
model.coef_standard_error_

array([ 0.01853424,  0.02674626,  0.02506636])

In [31]:
mod = statsmodels.discrete.discrete_model.Poisson(y_poisson, X)
res = mod.fit()
print(res.summary())

Optimization terminated successfully.
         Current function value: 1.588661
         Iterations 7
                          Poisson Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        Poisson   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Mon, 11 Sep 2017   Pseudo R-squ.:                  0.1932
Time:                        21:49:43   Log-Likelihood:                -15887.
converged:                       True   LL-Null:                       -19690.
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9796      0.019     52.851      0.000       0.943       1.016
x1            -1.9731      0.

In [32]:
s = Simulation(model)

In [33]:
s.sample(X, n_sim=10)

array([[ 4.,  3.,  2., ...,  2.,  0.,  0.],
       [ 4.,  4.,  1., ...,  0.,  1.,  0.],
       [ 3.,  4.,  2., ...,  1.,  0.,  1.],
       ..., 
       [ 3.,  2.,  4., ...,  2.,  1.,  0.],
       [ 3.,  3.,  1., ...,  1.,  1.,  2.],
       [ 2.,  4.,  2., ...,  1.,  1.,  3.]])

In [34]:
for model in s.parametric_bootstrap(X, n_sim=10):
    print(model.coef_)

[ 0.97584518 -1.95183053  1.00499433]
[ 0.96702939 -1.92501889  0.98326727]
[ 0.9531297  -1.91046988  1.01016099]
[ 0.96148266 -1.91118668  0.9859655 ]
[ 0.95810702 -1.92430305  1.0074289 ]
[ 0.97417409 -1.97883868  1.01112315]
[ 0.97400672 -1.97751903  0.99597901]
[ 0.97716044 -2.00407414  1.0068708 ]
[ 1.00623659 -1.97022556  0.97280256]
[ 0.98472008 -2.00665999  1.00411827]


In [35]:
for model in s.non_parametric_bootstrap(X, y_poisson, n_sim=10):
    print(model.coef_)

[ 0.93150692 -1.90839622  1.03641432]
[ 0.99669994 -1.95088065  0.95601371]
[ 0.9805434  -1.96057206  0.99846077]
[ 0.95466906 -1.9752257   1.03740027]
[ 0.969182   -1.9831375   1.01880549]
[ 0.99041008 -1.99963129  1.02092057]
[ 0.97968834 -1.92736145  0.96676223]
[ 0.95554389 -1.93148891  1.02090681]
[ 0.94413817 -1.92941545  1.0218936 ]
[ 0.96905418 -1.96604021  0.98990285]


## Poisson with Exposures

In [36]:
mu = np.exp(nu)
expos = np.random.uniform(0, 10, size=N)
y_poisson = np.random.poisson(lam=(mu*expos), size=N)

In [37]:
model = GLM(family=Poisson())
model.fit(X, y_poisson, offset=np.log(expos))

In [38]:
model.coef_

array([ 1.01089083, -2.01026895,  0.99239881])

In [39]:
model.coef_standard_error_

array([ 0.00819943,  0.01191424,  0.01115221])

## Gamma Regression

In [40]:
mu = np.exp(nu)
y_gamma = np.random.gamma(shape=2.0, scale=(mu / 2.0), size=N)

In [41]:
gamma_model = GLM(family=Gamma())
gamma_model.fit(X, y_gamma)

In [42]:
gamma_model.coef_

array([ 1.01040171, -1.99970979,  0.99452562])

In [43]:
gamma_model.coef_standard_error_

array([ 0.01917737,  0.02505722,  0.02526353])

In [44]:
gamma_model.dispersion_

0.53290114261870103

In [45]:
gamma_model = sm.GLM(y_gamma, X, 
                     family=sm.families.Gamma(
                         link=statsmodels.genmod.families.links.log))
res = gamma_model.fit()
print(res.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                   Gamma   Df Model:                            2
Link Function:                    log   Scale:                  0.501783523702
Method:                          IRLS   Log-Likelihood:                -13867.
Date:                Mon, 11 Sep 2017   Deviance:                       5327.4
Time:                        21:49:43   Pearson chi2:                 5.02e+03
No. Iterations:                     5                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0104      0.019     54.296      0.000       0.974       1.047
x1            -1.9997      0.024    -82.243      0.0

## Exponential Regression

In [46]:
mu = np.exp(nu)
y_exponential = np.random.exponential(scale=mu, size=N)

In [47]:
exponential_model = GLM(family=Gamma())
exponential_model.fit(X, y_exponential)

In [48]:
exponential_model.coef_

array([ 0.97932024, -1.9649605 ,  1.03097534])

In [49]:
exponential_model.coef_standard_error_

array([ 0.02792238,  0.03648348,  0.03678386])

In [50]:
exponential_model.dispersion_

1.1297264550109525

In [51]:
exponential_model = sm.GLM(y_exponential, X, 
                     family=sm.families.Gamma(
                         link=statsmodels.genmod.families.links.log))
res = exponential_model.fit()
print(res.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                   Gamma   Df Model:                            2
Link Function:                    log   Scale:                  0.985270846483
Method:                          IRLS   Log-Likelihood:                -15109.
Date:                Mon, 11 Sep 2017   Deviance:                       11294.
Time:                        21:49:43   Pearson chi2:                 9.85e+03
No. Iterations:                     6                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9793      0.026     37.556      0.000       0.928       1.030
x1            -1.9650      0.034    -57.672      0.0

## Linear Model with Correlated Predictors

In [52]:
N = 1000
X = np.empty(shape=(N, 3))
X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=N)
X[:, 2] = 0.5*X[:, 1] + np.random.uniform(-0.5, 0.5, size=N)
nu = 1 - 2*X[:, 1] + X[:, 2]

In [53]:
y = nu + np.random.normal(size=N)
model = GLM(family=Gaussian())
model.fit(X, y)

In [54]:
model.coef_

array([ 0.98412515, -1.92541664,  0.97230506])

In [55]:
model.coef_covariance_matrix_

array([[ 0.00436578, -0.00654351, -0.00011775],
       [-0.00654351,  0.01650181, -0.00650801],
       [-0.00011775, -0.00650801,  0.01309884]])

In [56]:
model.coef_standard_error_

array([ 0.06607408,  0.12845937,  0.11445019])

In [57]:
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.187
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                     114.5
Date:                Mon, 11 Sep 2017   Prob (F-statistic):           1.75e-45
Time:                        21:49:43   Log-Likelihood:                -1455.8
No. Observations:                1000   AIC:                             2918.
Df Residuals:                     997   BIC:                             2932.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9841      0.066     14.894      0.0